v1:all  :0.6940989850095997
    
v2: only ratings related + labels : 0.6931156236843117

v3: Top4 + labels: 0.6931116756196194

v4: Top4 + extra_features: 0.6849884392165065

v5: v4+ num_comments + release_year: 0.6819791322128397

v6: v5+ user_based_features: 0.6788406664932616

v7: v6 + RandomForest: 0.6759067263200159

In [52]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import KernelPCA
from sklearn.decomposition import PCA

from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings("ignore")

In [53]:
data = pd.read_csv('edited_train.csv')
data.loc[data['language'].isnull(), 'language'] = 'eng'

save_for_later = pd.read_csv('save_for_later.csv')
song_labels = pd.read_csv('song_labels.csv')

train, val = train_test_split(data, test_size=0.05, random_state=42, stratify = data['score'])
data.head()

,customer_id,song_id,score,First_Customer_id,platform_id,released_year,language,number_of_comments,First_song_id,song_avg,cust_avg
0,O29219,3459,3,O,P49540,1782.0,eng,1066.0,P,4.069767,3.029851
1,O29219,12,1,O,W13335037,2011.0,eng,101023.0,W,4.093750,3.029851
2,O29219,18,3,O,U5,1999.0,eng,36099.0,U,4.421668,3.029851
3,O29219,17,3,O,X6148028,2009.0,eng,88538.0,X,4.152371,3.029851
4,O29219,332,3,O,U65605,1953.0,eng,8690.0,U,3.863636,3.029851


In [54]:
save_for_later['count'] = 1
save_for_later.head()

,customer_id,song_id,count
0,F22180,1140,1
1,N34498,1523,1
2,K46875,6918,1
3,L50176,5828,1
4,F13830,3039,1


In [55]:
song_labels = song_labels.drop_duplicates(subset=['label_id', 'platform_id'], keep='last')
song_labels = song_labels.pivot(index='platform_id', columns='label_id', values='count')
save_for_later_user = save_for_later.pivot(index='customer_id', columns='song_id', values='count')
save_for_later_song = save_for_later.pivot(index='song_id', columns='customer_id', values='count')

train_pivot_user = train.pivot(index='customer_id', columns='song_id', values='score')
train_pivot_song = train.pivot(index='song_id', columns='customer_id', values='score')


X  = np.nan_to_num(song_labels, nan = 0)
pca = PCA(n_components=100, random_state = 42)
X = pca.fit_transform(X)
song_labels = pd.DataFrame(X, index = song_labels.index)

X  = np.nan_to_num(save_for_later_user, nan = 0)
pca = PCA(n_components=19, random_state = 42)
X = pca.fit_transform(X)
save_for_later_user = pd.DataFrame(X, index = save_for_later_user.index)

X  = np.nan_to_num(save_for_later_song, nan = 0)
pca = PCA(n_components=24, random_state = 42)
X = pca.fit_transform(X)
save_for_later_song = pd.DataFrame(X, index = save_for_later_song.index)


X  = np.nan_to_num(train_pivot_user, nan = 0)
pca = PCA(n_components=50, random_state = 42)
X = pca.fit_transform(X)
train_pivot_user = pd.DataFrame(X, index = train_pivot_user.index)

X  = np.nan_to_num(train_pivot_song, nan = 0)
pca = PCA(n_components=71, random_state = 42)
X = pca.fit_transform(X)
train_pivot_song = pd.DataFrame(X, index = train_pivot_song.index)

In [56]:
def merge_feature(train, df, feat_m, feat_trans, name, func, test = False):
    if func == 1:
        edited = train.groupby([feat_m]).count()
    if func == 2:
        edited = train.groupby([feat_m]).var()
    if func == 3:
        edited = train.groupby([feat_m]).std()
    if func == 4:
        edited = train.groupby([feat_m]).mean()
        
    edited = edited.reset_index()
    df = df.merge(edited[[feat_m, feat_trans]], how = 'left' , on = feat_m)
    if test == False:
        df = df.rename(columns ={feat_trans + '_x': feat_trans, feat_trans+'_y': name})
    else:
        df = df.rename(columns ={feat_trans : name})
    

    return df

In [57]:
train = merge_feature(train, train, 'customer_id', 'song_id','cust_count', 1)
val = merge_feature(train, val, 'customer_id', 'song_id','cust_count', 1)
train = merge_feature(train, train, 'customer_id', 'score','cust_var', 2)
val = merge_feature(train, val, 'customer_id', 'score','cust_var', 2)
train = merge_feature(train, train, 'customer_id', 'score','cust_std', 3)
val = merge_feature(train, val, 'customer_id', 'score','cust_std', 3)
# train = merge_feature(save_for_later, train, 'customer_id', 'song_id','later_usercount', 1)
# val = merge_feature(save_for_later, val, 'customer_id', 'song_id','later_usercount', 1)


train = merge_feature(train, train, 'song_id', 'customer_id','song_count', 1)
val = merge_feature(train, val, 'song_id', 'customer_id','song_count', 1)
train = merge_feature(train, train,'song_id', 'score','song_var', 2)
val = merge_feature(train, val, 'song_id', 'score','song_var', 2)
train = merge_feature(train, train,'song_id', 'score','song_std', 3)
val = merge_feature(train, val, 'song_id', 'score','song_std', 3)
# train = merge_feature(save_for_later, train, 'song_id', 'customer_id','later_songcount', 1)
# val = merge_feature(save_for_later, val, 'song_id', 'customer_id','later_songcount', 1)
# train = merge_feature(song_labels, train, 'platform_id', 'label_id','label_count', 1, True)
# val = merge_feature(song_labels, val, 'platform_id', 'label_id','label_count', 1, True)
# train = merge_feature(song_labels, train, 'platform_id', 'count','label_avg', 4, True)
# val = merge_feature(song_labels, val, 'platform_id', 'count','label_avg', 4, True)



train.head()

,customer_id,song_id,score,First_Customer_id,platform_id,released_year,language,number_of_comments,First_song_id,song_avg,cust_avg,cust_count,cust_var,cust_std,song_count,song_var,song_std
0,M41967,2621,4,M,P10890,1999.0,en-CA,2197.0,P,4.109091,3.529412,80,0.405063,0.636446,50,0.833061,0.912722
1,O8379,718,3,O,U10975,1929.0,eng,5485.0,U,3.706667,3.633333,59,0.582700,0.763348,216,1.462446,1.209316
2,J24604,7331,4,J,P5093760,2009.0,eng,1089.0,P,3.846154,4.052632,54,0.312020,0.558587,13,0.641026,0.800641
3,M1847,621,4,M,R37732,1970.0,eng,5168.0,R,3.886447,4.358491,51,0.323137,0.568452,257,1.012828,1.006394
4,H4872,244,2,H,Y66559,2006.0,en-US,28268.0,Y,3.734491,3.705882,64,0.586310,0.765709,385,0.815043,0.902797


In [58]:
train.shape

(673731, 17)

In [59]:
Y_train = train.score
# X_train = train.drop(['score','customer_id', 'song_id', 'platform_id' ,'language'], axis =1)
# X_test = val.drop(['score','customer_id', 'song_id', 'platform_id','language'], axis =1)
X_train = train[['song_avg', 'cust_avg' ,'cust_var','song_var', 'platform_id', 'customer_id', 
                 'number_of_comments', 'released_year', 'song_id']]
X_test = val[['song_avg', 'cust_avg' ,'cust_var','song_var', 'platform_id', 'customer_id', 
                 'number_of_comments', 'released_year', 'song_id']]
# cat_attribs = ['First_Customer_id', 'First_song_id']
# full_pipeline = ColumnTransformer([('cat', OneHotEncoder(handle_unknown='ignore'), cat_attribs)], remainder='passthrough')

# encoder = full_pipeline.fit(X_train)
# X_train = encoder.transform(X_train)
# X_test = encoder.transform(X_test)

In [60]:
X_train.shape

(673731, 9)

In [61]:
# song_labels = song_labels.reset_index()
X_train = X_train.merge(song_labels, how = 'left' , on = 'platform_id')
X_train = X_train.drop(['platform_id'],axis=1)

X_train = X_train.merge(save_for_later_user, how = 'left' , on = 'customer_id')
X_train = X_train.merge(train_pivot_user, how = 'left' , on = 'customer_id')
X_train = X_train.drop(['customer_id'],axis=1)

X_train = X_train.merge(save_for_later_song, how = 'left' , on = 'song_id')
X_train = X_train.merge(train_pivot_song, how = 'left' , on = 'song_id')
X_train = X_train.drop(['song_id'],axis=1)


X_train.shape

(673731, 270)

In [62]:
X_test = X_test.merge(song_labels, how = 'left' , on = 'platform_id')
X_test = X_test.drop(['platform_id'],axis=1)

X_test = X_test.merge(save_for_later_user, how = 'left' , on = 'customer_id')
X_test = X_test.merge(train_pivot_user, how = 'left' , on = 'customer_id')
X_test = X_test.drop(['customer_id'],axis=1)

X_test = X_test.merge(save_for_later_song, how = 'left' , on = 'song_id')
X_test = X_test.merge(train_pivot_song, how = 'left' , on = 'song_id')
X_test = X_test.drop(['song_id'],axis=1)

X_train.columns = list(range(len(X_train.columns)))
X_test.columns = list(range(len(X_test.columns)))

In [63]:
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

model =  CatBoostRegressor(iterations=2,
                          learning_rate=1,
                          depth=2, random_state=42)
model.fit(X_train, Y_train)

0:	learn: 0.8983314	total: 52.4ms	remaining: 52.4ms
1:	learn: 0.8769113	total: 90.4ms	remaining: 0us


In [64]:
preds = model.predict(X_test)
Y_test = val.score

In [65]:
PRED = np.around(preds,1)

PRED = np.clip(PRED, a_min# SUBMISSION
 = 1, a_max = 5)

In [66]:
mean_squared_error(Y_test, PRED)

0.7644686971235195

In [67]:
model1 =  xgb.XGBRegressor(learning_rate = 0.1)
model1.fit(X_train, Y_train)

preds1 = model1.predict(X_test)
Y_test = val.score

PRED1 = np.around(preds1,1)

PRED1 = np.clip(PRED1, a_min# SUBMISSION
 = 1, a_max = 5)

PRED_final = (PRED + PRED1)/2

mean_squared_error(Y_test, PRED_final)

0.7000320080114663

In [87]:
mean_squared_error(Y_test, PRED)

0.6788406664932616

In [133]:
train = pd.read_csv('edited_train.csv')
val = pd.read_csv('edited_test.csv') 
test = pd.read_csv('test.csv')


print(val.shape,train.shape)

(670681, 10) (709191, 11)


In [134]:
train = merge_feature(train, train, 'customer_id', 'song_id','cust_count', 1)
val = merge_feature(train, val, 'customer_id', 'song_id','cust_count', 1)
train = merge_feature(train, train, 'customer_id', 'score','cust_var', 2)
val = merge_feature(train, val, 'customer_id', 'score','cust_var', 2, True)
train = merge_feature(train, train, 'customer_id', 'score','cust_std', 3)
val = merge_feature(train, val, 'customer_id', 'score','cust_std', 3, True)
# train = merge_feature(save_for_later, train, 'customer_id', 'song_id','later_usercount', 1)
# val = merge_feature(save_for_later, val, 'customer_id', 'song_id','later_usercount', 1)



train = merge_feature(train, train, 'song_id', 'customer_id','song_count', 1)
val = merge_feature(train, val, 'song_id', 'customer_id','song_count', 1)
train = merge_feature(train, train,'song_id', 'score','song_var', 2)
val = merge_feature(train, val, 'song_id', 'score','song_var', 2, True)
train = merge_feature(train, train,'song_id', 'score','song_std', 3)
val = merge_feature(train, val, 'song_id', 'score','song_std', 3, True)
# train = merge_feature(save_for_later, train, 'song_id', 'customer_id','later_songcount', 1)
# val = merge_feature(save_for_later, val, 'song_id', 'customer_id','later_songcount', 1)

val.head()

,customer_id,song_id,First_Customer_id,platform_id,released_year,language,number_of_comments,First_song_id,song_avg,cust_avg,cust_count,cust_var,cust_std,song_count,song_var,song_std
0,J24644,5304.0,J,U64955,1989.0,NaN,416.0,U,3.468085,3.566667,60,0.859887,0.927301,47.0,0.689177,0.830167
1,J24644,1423.0,J,Y10619,1995.0,eng,1714.0,Y,3.428571,3.566667,60,0.859887,0.927301,126.0,1.014857,1.007401
2,J24644,670.0,J,S11573,1979.0,en-US,2294.0,S,3.843137,3.566667,60,0.859887,0.927301,255.0,0.676085,0.822244
3,J24644,4465.0,J,V653396,1987.0,en-US,672.0,V,3.692308,3.566667,60,0.859887,0.927301,39.0,1.218623,1.103913
4,J24644,5.0,J,Q4671,1925.0,eng,51992.0,Q,3.846195,3.566667,60,0.859887,0.927301,1879.0,1.049281,1.024344


In [135]:
Y_train = train.score
# X_train = train.drop(['score','customer_id', 'song_id', 'platform_id' ,'language'], axis =1)
# X_test = val.drop(['score','customer_id', 'song_id', 'platform_id','language'], axis =1)
X_train = train[['song_avg', 'cust_avg' ,'cust_var','song_var', 'platform_id', 'customer_id']]
X_test = val[['song_avg', 'cust_avg' ,'cust_var','song_var', 'platform_id', 'customer_id']]

X_train = X_train.merge(song_labels, how = 'left' , on = 'platform_id')
X_train = X_train.drop(['platform_id'],axis=1)

X_train = X_train.merge(save_for_later, how = 'left' , on = 'customer_id')
X_train = X_train.merge(train_pivot, how = 'left' , on = 'customer_id')
X_train = X_train.drop(['customer_id'],axis=1)


X_test = X_test.merge(song_labels, how = 'left' , on = 'platform_id')
X_test = X_test.drop(['platform_id'],axis=1)

X_test = X_test.merge(save_for_later, how = 'left' , on = 'customer_id')
X_test = X_test.merge(train_pivot, how = 'left' , on = 'customer_id')
X_test = X_test.drop(['customer_id'],axis=1)


model =  xgb.XGBRegressor(learning_rate = 0.1)
model.fit(X_train, Y_train)

preds = model.predict(X_test)
PRED = np.around(preds,1)

PRED = np.clip(PRED, a_min# SUBMISSION
 = 1, a_max = 5)

In [136]:
val['preds'] = PRED

In [137]:
test = test.merge(val[['customer_id','song_id','preds']], how = 'left', on = ['customer_id', 'song_id'])
test.head()

,customer_id,song_id,preds
0,J24644,5304,3.2
1,G29701,123,3.7
2,M29977,11,4.2
3,H50752,1445,3.6
4,J42394,271,4.5


In [138]:
PRED = test.preds
df_submission = pd.read_csv('dummy_submission.csv')
df_submission.score = PRED
df_submission.to_csv('./RL_v4.csv',index=False)

In [139]:
PRED.size

670681

In [140]:
pred1 = pd.read_csv('RL_v4.csv')
pred2 = pd.read_csv('RM_v4.csv')

pred1['score']  = 0.5*(pred1['score'] + pred2['score'])
pred1.head()

,test_row_id,score
0,0,3.30
1,1,3.85
2,2,4.10
3,3,3.75
4,4,4.30


In [141]:
pred1.to_csv('RL_RM_v44', index = False)